# Preprocess Data
- Read price and fundamental data and save as dataframe
- Clean and process data
- Combine these two dataframe
- Calculate percent difference
- Export data into Excel file

In [122]:
# 600188.SH 兖州煤业

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Series,DataFrame 


### Get price data from Excel file

In [123]:
df_price = pd.read_excel('price_data.xlsx')

# set date as index
def setting_index_price(df):
    """
    Returns a sorted datetime index with monthly frequency
    """
    df['date'] = pd.to_datetime(df['date'])
    df.set_index("date", inplace=True)
    df['date'] = df.index.to_period('M')
    df.drop_duplicates('date', keep='last', inplace=True)
    df.set_index("date", inplace=True)
    # deal with NaN data
    df = df.fillna(method='ffill')
    # get specific columns we need
    df = df.loc[:, ['high', 'low']]
    df.sort_index(ascending=True, inplace=True)
    return df

df_price = setting_index_price(df_price)

In [124]:
df_price.head()

,high,low
date,,
2000-02,7.79,7.40
2000-03,10.52,9.90
2000-04,10.08,9.69
2000-05,9.79,9.31
2000-06,10.24,9.88


### Get fundamental data from Excel file

In [125]:
df_funda = pd.read_excel('fundamental_data.xlsx')

# set date as index
def setting_index_fundamental(df):
    """
    Returns a sorted datetime index with month frequency
    """
    df.set_index("catalogue", inplace=True)
    df = df.T
    df['date'] = pd.to_datetime(df.index)
    df.set_index("date", inplace=True)
    df['date'] = df.index.to_period('M')
    df.drop_duplicates('date', keep='last', inplace=True)
    df.set_index("date", inplace=True)
    # deal with NaN data
    df = df.fillna(method='ffill')
    df.sort_index(ascending=True, inplace=True)
    return df

df_funda = setting_index_fundamental(df_funda)

In [95]:
df_funda.head()

catalogue,营业总收入,营业收入,营业总成本,销售费用,管理费用,财务费用,投资净收益,营业利润,净利润,归属于母公司所有者的净利润,基本每股收益,总股本（万股）,总资产,流动资产,总负债,流动负债,少数股东权益,无形资产,商誉
date,,,,,,,,,,,,,,,,,,,
2002-12,2112123.997,2112123.997,1874723.715,398812.830,390322.766,5068.280,3687.968,246642.076,160758.936,160515.251,0.0559,2870000.0,1.298355e+07,3897273.730,3320038.891,1944259.784,4936.449,777436.629,0.0
2003-03,2007673.447,2007673.447,1615384.114,375832.455,257389.695,16095.229,0.000,403986.871,259240.516,259213.850,0.0903,2870000.0,1.332590e+07,4427880.597,3339050.702,1920060.964,4367.202,771485.451,0.0
2003-06,2420206.371,2420206.371,1969433.658,528020.587,341853.476,12027.327,1478.333,473833.772,318358.349,317556.661,0.1106,2870000.0,1.282320e+07,4032762.607,2452823.821,1618354.131,3999.263,802999.924,0.0
2003-09,2156975.818,2156975.818,1733074.761,389842.771,246555.089,6950.704,5998.113,432243.915,281770.268,281746.701,0.0982,2870000.0,1.299395e+07,4015167.564,2278985.517,1428195.124,4022.830,795830.858,0.0
2003-12,2080376.568,2080376.568,1709880.282,365378.978,342814.945,6297.159,2940.525,374834.025,246517.586,246847.435,0.0860,2870000.0,1.393860e+07,4490526.096,2911247.621,2346738.239,3739.101,788638.696,0.0


### Combine price and fundamental dataframe into a joined dataframe
Converting all values to numeric values

In [126]:
joined_df = df_funda.join(df_price)
joined_df = joined_df.apply(pd.to_numeric)
joined_df.head()

,gross revenue,revenue,total operating cost,selling expense,administration expense,financial expense,net investment income,operating profit,retained profit,net income attributable to parent company,...,"total share (10,000 shares)",total assets,current assets,total liabilities,current liabilities,minority equity,intangible assets,goodwill,high,low
date,,,,,,,,,,,,,,,,,,,,,
2002-12,2112123.997,2112123.997,1874723.715,398812.830,390322.766,5068.280,3687.968,246642.076,160758.936,160515.251,...,2870000.0,1.298355e+07,3897273.730,3320038.891,1944259.784,4936.449,777436.629,0.0,8.30,8.01
2003-03,2007673.447,2007673.447,1615384.114,375832.455,257389.695,16095.229,0.000,403986.871,259240.516,259213.850,...,2870000.0,1.332590e+07,4427880.597,3339050.702,1920060.964,4367.202,771485.451,0.0,9.15,8.81
2003-06,2420206.371,2420206.371,1969433.658,528020.587,341853.476,12027.327,1478.333,473833.772,318358.349,317556.661,...,2870000.0,1.282320e+07,4032762.607,2452823.821,1618354.131,3999.263,802999.924,0.0,9.24,8.88
2003-09,2156975.818,2156975.818,1733074.761,389842.771,246555.089,6950.704,5998.113,432243.915,281770.268,281746.701,...,2870000.0,1.299395e+07,4015167.564,2278985.517,1428195.124,4022.830,795830.858,0.0,7.78,7.67
2003-12,2080376.568,2080376.568,1709880.282,365378.978,342814.945,6297.159,2940.525,374834.025,246517.586,246847.435,...,2870000.0,1.393860e+07,4490526.096,2911247.621,2346738.239,3739.101,788638.696,0.0,11.23,10.82


### Replace values with percent difference 
Between each quarter  
Mutliply by 100 for better readability

In [127]:
pcnt_df = joined_df.pct_change(periods=1).apply(lambda x: x*100)
pcnt_df = pcnt_df.drop(pcnt_df.index[0])


#### Replace infinite values with NaN

In [128]:
pcnt_df = pcnt_df.replace([np.inf, -np.inf], np.nan)


## Create the trade strategy
- Buy (if the highest high and lowest low of the quarter will both increase by 3% or more in next quarter)
- Sell (if the lowest low and highest high of the quarter will both decrease by 3% or more in next quarter)
- Hold (if it will not do either in next quarter)

In [129]:
def class_creation(df, thres=3):
    """
    - sell(0)
    - buy(1)
    - hold(2)
    
    Threshold can be changed to fit the performance of market and specific stock
    """
    if df['high'] <= -thres and df['low'] <= -thres:
        return 0
    
    elif df['high'] >= thres and df['low'] >= thres:
        return 1
    
    else:
        return 2

Creating a new DataFrame that is added the column 'Decision' showing if a quarterly reports indicate us to buy, hold, or sell.  
- Why is dataframe shifted by -1?  
We should know if the prices will increase/decrease in the next quarter.   
In other word, the decision which is indicated by this quarter should be made in last quarter.   

In [130]:
decision_df = pcnt_df
    
decision_df['Decision'] = decision_df.apply(class_creation, axis=1).shift(-1)
decision_df = decision_df[0:-1]
decision_df.head()

,gross revenue,revenue,total operating cost,selling expense,administration expense,financial expense,net investment income,operating profit,retained profit,net income attributable to parent company,...,total assets,current assets,total liabilities,current liabilities,minority equity,intangible assets,goodwill,high,low,Decision
date,,,,,,,,,,,,,,,,,,,,,
2003-03,-4.945285,-4.945285,-13.833484,-5.762196,-34.057217,217.567873,-100.000000,63.794790,61.260408,61.488611,...,2.636775,13.614822,0.572638,-1.244629,-11.531508,-0.765487,NaN,10.240964,9.987516,2.0
2003-06,20.547810,20.547810,21.917360,40.493611,32.815526,-25.273962,NaN,17.289399,22.804241,22.507598,...,-3.772301,-8.923411,-26.541283,-15.713399,-8.425051,4.084908,NaN,0.983607,0.794552,0.0
2003-09,-10.876368,-10.876368,-12.001364,-26.169021,-27.876969,-42.209071,305.734905,-8.777310,-11.492735,-11.276715,...,1.331521,-0.436302,-7.087272,-11.750148,0.589284,-0.892785,NaN,-15.800866,-13.626126,1.0
2003-12,-3.551234,-3.551234,-1.338343,-6.275297,39.041926,-9.402573,-50.975832,-13.281827,-12.511143,-12.386752,...,7.269940,11.839071,27.743138,64.314959,-7.052970,-0.903730,NaN,44.344473,41.069100,1.0
2004-03,12.996168,12.996168,2.929985,-20.751992,9.729352,-29.727009,-57.911291,60.846788,59.022115,58.765778,...,2.888365,14.419261,-1.776355,-2.416558,2.894519,-0.910106,NaN,32.235085,32.902033,0.0


#### Quick check for NaN values

In [131]:
decision_df.isna().sum()

gross revenue                                 0
revenue                                       0
total operating cost                          0
selling expense                               0
administration expense                        0
financial expense                             0
net investment income                         3
operating profit                              0
retained profit                               0
net income attributable to parent company     0
basic EPS                                     0
total share (10,000 shares)                   0
total assets                                  0
current assets                                0
total liabilities                             0
current liabilities                           0
minority equity                               0
intangible assets                             0
goodwill                                     16
high                                          0
low                                     

#### Filling the NaNs

In [132]:
decision_df = decision_df.fillna(method='ffill')
# deal with goodwill, replace earliest some goodwill with 0
decision_df = decision_df.fillna(value=0)
decision_df[np.isinf(decision_df)] = 0
final_df = decision_df

#### Quick check for NaN values again

In [133]:
final_df.isna().sum()
np.isinf(final_df).sum()

gross revenue                                0
revenue                                      0
total operating cost                         0
selling expense                              0
administration expense                       0
financial expense                            0
net investment income                        0
operating profit                             0
retained profit                              0
net income attributable to parent company    0
basic EPS                                    0
total share (10,000 shares)                  0
total assets                                 0
current assets                               0
total liabilities                            0
current liabilities                          0
minority equity                              0
intangible assets                            0
goodwill                                     0
high                                         0
low                                          0
Decision     

### Resetting the index
We no longer need the dates because we have calculated percent difference.

In [134]:
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,gross revenue,revenue,total operating cost,selling expense,administration expense,financial expense,net investment income,operating profit,retained profit,net income attributable to parent company,...,total assets,current assets,total liabilities,current liabilities,minority equity,intangible assets,goodwill,high,low,Decision
0,-4.945285,-4.945285,-13.833484,-5.762196,-34.057217,217.567873,-100.000000,63.794790,61.260408,61.488611,...,2.636775,13.614822,0.572638,-1.244629,-11.531508,-0.765487,0.0,10.240964,9.987516,2.0
1,20.547810,20.547810,21.917360,40.493611,32.815526,-25.273962,-100.000000,17.289399,22.804241,22.507598,...,-3.772301,-8.923411,-26.541283,-15.713399,-8.425051,4.084908,0.0,0.983607,0.794552,0.0
2,-10.876368,-10.876368,-12.001364,-26.169021,-27.876969,-42.209071,305.734905,-8.777310,-11.492735,-11.276715,...,1.331521,-0.436302,-7.087272,-11.750148,0.589284,-0.892785,0.0,-15.800866,-13.626126,1.0
3,-3.551234,-3.551234,-1.338343,-6.275297,39.041926,-9.402573,-50.975832,-13.281827,-12.511143,-12.386752,...,7.269940,11.839071,27.743138,64.314959,-7.052970,-0.903730,0.0,44.344473,41.069100,1.0
4,12.996168,12.996168,2.929985,-20.751992,9.729352,-29.727009,-57.911291,60.846788,59.022115,58.765778,...,2.888365,14.419261,-1.776355,-2.416558,2.894519,-0.910106,0.0,32.235085,32.902033,0.0


### Drop the Price columns
- Price high
- Price low

To prevent any data leakage because we are looking mainly at the relationship of value changes of fundamental data and trade decision rather than prices themselves.

In [135]:
final_df.drop(['high', 'low'], 1, inplace=True)

### Counting how many classes of trade decision there are
Unequal classes are fine because we will use a specific evaluation metric to determine success in classification.

In [136]:
final_df['Decision'].value_counts()

1.0    29
0.0    27
2.0    10
Name: Decision, dtype: int64

### Export the final dataframe

In [137]:
final_df.to_excel('final_data.xlsx')